In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from obspy.fdsn import Client
from obspy import UTCDateTime
from StringIO import StringIO
import re
from obspy import readEvents
from mpl_toolkits.basemap import Basemap
from obspy.imaging.beachball import Beach
import matplotlib.cm as cm
import datetime
import webbrowser  
import math
from matplotlib.lines import Line2D
from obspy import UTCDateTime
print 'Done imports'

Done imports


In [2]:
def plot_event(catalog, projection='cyl', resolution='l',
             continent_fill_color='0.9',
             water_fill_color='white',
             label='magnitude',
             color='depth',
             colormap=None, us=False, **kwargs):  # @UnusedVariable
        """
        Creates preview map of all events in current Catalog object.

        :type projection: str, optional
        :param projection: The map projection. Currently supported are
            * ``"cyl"`` (Will plot the whole world.)
            * ``"ortho"`` (Will center around the mean lat/long.)
            * ``"local"`` (Will plot around local events)
            Defaults to "cyl"
        :type resolution: str, optional
        :param resolution: Resolution of the boundary database to use. Will be
            based directly to the basemap module. Possible values are
            * ``"c"`` (crude)
            * ``"l"`` (low)
            * ``"i"`` (intermediate)
            * ``"h"`` (high)
            * ``"f"`` (full)
            Defaults to ``"l"``
        :type continent_fill_color: Valid matplotlib color, optional
        :param continent_fill_color:  Color of the continents. Defaults to
            ``"0.9"`` which is a light gray.
        :type water_fill_color: Valid matplotlib color, optional
        :param water_fill_color: Color of all water bodies.
            Defaults to ``"white"``.
        :type label: str, optional
        :param label:Events will be labeld based on the chosen property.
            Possible values are
            * ``"magnitude"``
            * ``None``
            Defaults to ``"magnitude"``
        :type color: str, optional
        :param color:The events will be color-coded based on the chosen
            proberty. Possible values are
            * ``"date"``
            * ``"depth"``
            Defaults to ``"depth"``
        :type colormap: str, optional, any matplotlib colormap
        :param colormap: The colormap for color-coding the events.
            The event with the smallest property will have the
            color of one end of the colormap and the event with the biggest
            property the color of the other end with all other events in
            between.
            Defaults to None which will use the default colormap for the date
            encoding and a colormap going from green over yellow to red for the
            depth encoding.

        .. rubric:: Example

        >>> cat = readEvents( \
            "http://www.seismicportal.eu/services/event/search?magMin=8.0") \
            # doctest:+SKIP
        >>> cat.plot()  # doctest:+SKIP
        """
        from mpl_toolkits.basemap import Basemap
        import matplotlib.pyplot as plt
        from matplotlib.colors import Normalize
        from matplotlib.cm import ScalarMappable
        import matplotlib as mpl

        if color not in ('date', 'depth'):
            raise ValueError('Events can be color coded by date or depth. '
                             "'%s' is not supported." % (color,))
        if label not in (None, 'magnitude', 'depth'):
            raise ValueError('Events can be labeled by magnitude or events can'
                             ' not be labeled. '
                             "'%s' is not supported." % (label,))

        # lat/lon coordinates, magnitudes, dates
        lats = []
        lons = []
        labels = []
        mags = []
        colors = []
        for i, event in enumerate(catalog):
            lats.append(event[0])
            lons.append(event[1])
            mag = np.float(event[3])
            mags.append(mag)
            labels.append(('    %.1f' % mag) if mag and label == 'magnitude'
                          else '')
            if color == 'date':
                colors.append(event[6])
            elif color == 'depth':
                colors.append(event[2])
        min_color = min(colors)
        max_color = max(colors)

        # Create the colormap for date based plotting.
        if colormap is None:
            if color == "date":
                colormap = plt.get_cmap()
            else:
                # Choose green->yellow->red for the depth encoding.
                colormap = plt.get_cmap("RdYlGn_r")
                
        scal_map = ScalarMappable(norm=Normalize(min_color, max_color),
                                  cmap=colormap)
        scal_map.set_array(np.linspace(0, 1, 1))

        fig = plt.figure(figsize=(16,24))
        # The colorbar should only be plotted if more then one event is
        # present.
        if len(catalog) > 1:
            map_ax = fig.add_axes([0.03, 0.13, 0.94, 0.82])
            #cm_ax = fig.add_axes([0.03, 0.05, 0.94, 0.05])
            #rect = [left, bottom, width, height]
            cm_ax = fig.add_axes([0.98, 0.39, 0.04, 0.3])
            plt.sca(map_ax)
        else:
            map_ax = fig.add_axes([0.05, 0.05, 0.90, 0.90])

        if projection == 'cyl':
            if us:
                map = Basemap(lat_0=38, lon_0=-122.0,resolution ='h',
                          llcrnrlon=-125,llcrnrlat=20,urcrnrlon=-60,urcrnrlat=60)
            else:
                map = Basemap(resolution=resolution)
        elif projection == 'ortho':
            map = Basemap(projection='ortho', resolution=resolution,
                          area_thresh=1000.0, lat_0=sum(lats) / len(lats),
                          lon_0=sum(lons) / len(lons))
        elif projection == 'local':
            if min(lons) < -150 and max(lons) > 150:
                max_lons = max(np.array(lons) % 360)
                min_lons = min(np.array(lons) % 360)
            else:
                max_lons = max(lons)
                min_lons = min(lons)
            lat_0 = (max(lats) + min(lats)) / 2.
            lon_0 = (max_lons + min_lons) / 2.
            if lon_0 > 180:
                lon_0 -= 360
            deg2m_lat = 2 * np.pi * 6371 * 1000 / 360
            deg2m_lon = deg2m_lat * np.cos(lat_0 / 180 * np.pi)
            if len(lats) > 1:
                height = (max(lats) - min(lats)) * deg2m_lat
                width = (max_lons - min_lons) * deg2m_lon
                margin = 0.2 * (width + height)
                height += margin
                width += margin
            else:
                height = 2.0 * deg2m_lat
                width = 5.0 * deg2m_lon
            
            map = Basemap(projection='aeqd', resolution=resolution,
                          area_thresh=1000.0, lat_0=lat_0, lon_0=lon_0,
                          width=width, height=height)
            # not most elegant way to calculate some round lats/lons

            def linspace2(val1, val2, N):
                """
                returns around N 'nice' values between val1 and val2
                """
                dval = val2 - val1
                round_pos = int(round(-np.log10(1. * dval / N)))
                delta = round(2. * dval / N, round_pos) / 2
                new_val1 = np.ceil(val1 / delta) * delta
                new_val2 = np.floor(val2 / delta) * delta
                N = (new_val2 - new_val1) / delta + 1
                return np.linspace(new_val1, new_val2, N)
            N1 = int(np.ceil(height / max(width, height) * 8))
            N2 = int(np.ceil(width / max(width, height) * 8))
            map.drawparallels(linspace2(lat_0 - height / 2 / deg2m_lat,
                                        lat_0 + height / 2 / deg2m_lat, N1),
                              labels=[0, 1, 1, 0])
            if min(lons) < -150 and max(lons) > 150:
                lon_0 %= 360
            meridians = linspace2(lon_0 - width / 2 / deg2m_lon,
                                  lon_0 + width / 2 / deg2m_lon, N2)
            meridians[meridians > 180] -= 360
            map.drawmeridians(meridians, labels=[1, 0, 0, 1])
        else:
            msg = "Projection %s not supported." % projection
            raise ValueError(msg)

        # draw coast lines, country boundaries, fill continents.
        map.drawcoastlines(color="0.4")
        map.drawcountries(color="0.75")
        map.fillcontinents(color=continent_fill_color,
                           lake_color=water_fill_color)
        # draw the edge of the map projection region (the projection limb)
        map.drawmapboundary(fill_color=water_fill_color)
        # draw lat/lon grid lines every 30 degrees.
        #map.drawmeridians(np.arange(-180, 180, 30),labels=[1,1,0,0])
        #map.drawparallels(np.arange(-90, 90, 30), labels=[0,0,1,1])
        map.drawparallels(np.arange(-90., 120., 30.), labels=[1,0,0,0])
        map.drawmeridians(np.arange(0., 360., 60.), labels=[0,0,0,1])

        # compute the native map projection coordinates for events.
        x, y = map(lons, lats)
        # plot labels
        if 100 > len(catalog) > 1:
            for name, xpt, ypt, colorpt in zip(labels, x, y, colors):
                # Check if the point can actually be seen with the current map
                # projection. The map object will set the coordinates to very
                # large values if it cannot project a point.
                if xpt > 1e25:
                    continue
                plt.text(xpt, ypt, name, weight="heavy",
                         color=scal_map.to_rgba(colorpt))
        elif len(catalog) == 1:
            plt.text(x[0], y[0], labels[0], weight="heavy", color="red")
        min_size = 6
        max_size = 30
        min_mag = min(mags)
        max_mag = max(mags)
        if len(catalog) > 1:
            frac = [(_i - min_mag) / (max_mag - min_mag) for _i in mags]
            magnitude_size = [(_i * (max_size - min_size)) ** 2 for _i in frac]
            #magnitude_size = [(_i * min_size) for _i in mags]
            #print magnitude_size
            colors_plot = [scal_map.to_rgba(c) for c in colors]
        else:
            magnitude_size = 15.0 ** 2
            colors_plot = "red"
        map.scatter(x, y, marker='o', s=magnitude_size, c=colors_plot,
                    zorder=10)
        times = [event[6] for event in catalog]
        if len(catalog) > 1:
            plt.title(
                "{event_count} events ({start} to {end}) "
                "- Color codes {colorcode}, size the magnitude".format(
                    event_count=len(catalog),
                    start=min(times).strftime("%Y-%m-%d"),
                    end=max(times).strftime("%Y-%m-%d"),
                    colorcode="origin time" if color == "date" else "depth"))
        else:
            plt.title("Event at %s" % times[0].strftime("%Y-%m-%d"))

        # Only show the colorbar for more than one event.
        if len(catalog) > 1:
            cb = mpl.colorbar.ColorbarBase(ax=cm_ax, cmap=colormap, orientation='vertical')
            cb.set_ticks([0, 0.25, 0.5, 0.75, 1.0])
            color_range = max_color - min_color
            cb.set_ticklabels([
                _i.strftime('%Y-%b-%d') if color == "date" else '%.1fkm' %
                (_i)
                for _i in [min_color, min_color + color_range * 0.25,
                           min_color + color_range * 0.50,
                           min_color + color_range * 0.75, max_color]])

        plt.show()

In [5]:
t1 = UTCDateTime("2014-09-14T00:00:00.000")
t2 = UTCDateTime("2014-10-23T00:00:00.000")

#get the catalog
client = Client('USGS')

sio = StringIO()
#save the catalog into a StringIO object
cat = client.get_events(starttime=t1, endtime=t2, minmagnitude=5,includeallorigins=True, filename=sio)

print cat

#specify the entries you want to replace with (the inconsistent ones) in the following dictionary
rep = {"quarry_blast": "quarry blast", "quarry": "quarry blast", "quarry blast_blast":"quarry blast" }

#replace the multiple entries, and save the modified entries into StringIO object
rep = dict((re.escape(k), v) for k, v in rep.iteritems())
pattern = re.compile("|".join(rep.keys()))

sio.seek(0)
sio2 = StringIO()
sio2.write(pattern.sub(lambda m: rep[re.escape(m.group(0))], sio.buf))

#read the catalog from this StringIO object
sio2.seek(0)
cat = readEvents(sio2)
print (cat)
print cat[1]

None
171 Event(s) in Catalog:
2014-10-22T21:56:55.990000Z | +14.951,  -44.941 | 5.3 mb | manual
2014-10-22T12:38:21.510000Z | -55.368,  -27.996 | 5.0 mb | manual
...
2014-09-14T16:34:22.650000Z |  +1.131,  +97.244 | 5.1 mb | manual
2014-09-14T04:52:26.940000Z |  +1.146,  +97.256 | 5.3 mb | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'
Event:	2014-10-22T12:38:21.510000Z | -55.368,  -27.996 | 5.0 mb | manual

	        resource_id: ResourceIdentifier(id="quakeml:earthquake.usgs.gov/fdsnws/event/1/query?eventid=usb000spqa&format=quakeml")
	         event_type: u'earthquake'
	      creation_info: CreationInfo(agency_id='us', creation_time=UTCDateTime(2015, 1, 17, 1, 25, 32, 40000))
	---------
	 event_descriptions: 1 Elements
	            origins: 1 Elements
	         magnitudes: 1 Elements


In [8]:
eq_matrix = []
evtime_mat = []
mt = []
event_id =[]

mt_type = 'Moment_tensor'
#'Focal'
#'Moment_tensor'
#'Both'

for index in range(cat.count()):
    myevent = cat[index]
    
    origins = myevent.origins[0]
    evtime_mat.append(origins.time)
    evla = origins.latitude
    evlo = origins.longitude
    evdp = origins.depth / 1000.
    #quality = origins.quality
    mag_type = myevent.magnitudes[0].magnitude_type
    mag = myevent.magnitudes[0].mag
    event_type = myevent.event_type
    
    print myevent.focal_mechanisms
    
    if myevent.focal_mechanisms != []:  
        
        if mt_type == 'Moment_tensor':
            moment_tensor = myevent.focal_mechanisms[0].moment_tensor.tensor
            if moment_tensor is not None:
                eventid = myevent['resource_id'].id.split('&')[0].split('=')[1]
                m_rr = moment_tensor.m_rr
                m_tt = moment_tensor.m_tt
                m_pp = moment_tensor.m_pp
                m_rt = moment_tensor.m_rt
                m_rp = moment_tensor.m_rp
                m_tp = moment_tensor.m_tp
                
                #m_rr=3.315e+16, m_tt=-6.189e+16, m_pp=2.874e+16, m_rt=-5311000000000000.0, m_rp=-1.653e+16, m_tp=5044000000000000.0
                mt.append([evla, evlo, evdp, mag, m_rr, m_tt,m_pp,m_rt,m_rp,m_tp])
                event_id.append(eventid)
        elif mt_type == 'Focal':
            nodal_plane = myevent.focal_mechanisms[0].nodal_planes.nodal_plane_1
            if nodal_plane is not None:
                eventid = myevent['resource_id'].id.split('&')[0].split('=')[1]
                mt.append([evla, evlo, evdp, mag, nodal_plane.strike, nodal_plane.dip, nodal_plane.rake])
                event_id.append(eventid)
        elif mt_type == 'Both':
            
            nodal_plane = myevent.focal_mechanisms[0].nodal_planes.nodal_plane_1
            if nodal_plane is not None:
                eventid = myevent['resource_id'].id.split('&')[0].split('=')[1]
                mt.append([evla, evlo, evdp, mag, nodal_plane.strike, nodal_plane.dip, nodal_plane.rake])
                event_id.append(eventid)
                break
            else:
                pass
            
            moment_tensor = myevent.focal_mechanisms[0].moment_tensor.tensor
            if moment_tensor is not None:
                eventid = myevent['resource_id'].id.split('&')[0].split('=')[1]
                m_rr = moment_tensor.m_rr
                m_tt = moment_tensor.m_tt
                m_pp = moment_tensor.m_pp
                m_rt = moment_tensor.m_rt
                m_rp = moment_tensor.m_rp
                m_tp = moment_tensor.m_tp
                
                #m_rr=3.315e+16, m_tt=-6.189e+16, m_pp=2.874e+16, m_rt=-5311000000000000.0, m_rp=-1.653e+16, m_tp=5044000000000000.0
                mt.append([evla, evlo, evdp, mag, m_rr, m_tt,m_pp,m_rt,m_rp,m_tp])
                event_id.append(eventid)
            

    eq_matrix.append((evla, evlo, evdp, mag, mag_type, event_type, origins.time))
    #print mt
    #evla = origins[1]

quarry_blast = [tmp for tmp in eq_matrix if tmp[5] == 'quarry blast']
earthquakes =  [tmp for tmp in eq_matrix if tmp[5] == 'earthquake']

evtime = [tmp.datetime for tmp in evtime_mat]
print eq_matrix
    
    

[]
[]
[]
[FocalMechanism(resource_id=ResourceIdentifier(id="quakeml:earthquake.usgs.gov/archive/product/moment-tensor/us_b000spn6_mwc_gcmt/us/1421457931040/product.xml"), nodal_planes=NodalPlanes(nodal_plane_1=NodalPlane(strike=38.0, dip=57.0, rake=89.0), nodal_plane_2=NodalPlane(strike=219.0, dip=33.0, rake=91.0)), evaluation_mode=u'manual', moment_tensor=MomentTensor(derived_origin_id=ResourceIdentifier(id="quakeml:earthquake.usgs.gov/archive/product/moment-tensor/us_b000spn6_mwc_gcmt/us/1421457931040/product.xml#origin"), moment_magnitude_id=ResourceIdentifier(id="quakeml:earthquake.usgs.gov/archive/product/moment-tensor/us_b000spn6_mwc_gcmt/us/1421457931040/product.xml#magnitude"), tensor=Tensor(m_rr=5e+17, m_tt=-1.82e+17, m_pp=-3.18e+17, m_rt=1.35e+17, m_rp=1.82e+17, m_tp=-2.85e+17), double_couple=0.8545), creation_info=CreationInfo(agency_id='gcmt', creation_time=UTCDateTime(2015, 1, 17, 1, 25, 31, 40000))), FocalMechanism(resource_id=ResourceIdentifier(id="quakeml:earthquake.usg

In [ ]:
from obspy.core.util import locations2degrees
def check_collision(lats, lons, radius, dist_bt):
    
    lats_m = np.zeros(len(lats))
    lons_m = np.zeros(len(lats))
    indicator = np.zeros(len(lats))
    
    angles = range(0,360,28)
    j = 0
    for i in range(len(lats)-1):
        for k in range(i+1, len(lats)):
            dist = locations2degrees(lats[i], lons[i], lats[k], lons[k]) * 111
            if dist < dist_bt:
                indicator[i] =1
                indicator[k] =1
            else:
                pass
    
    for i in range(len(lats)):
        if indicator[i] == 1:
    
            ix = j%len(angles)
            a = radius*np.cos(angles[ix]*math.pi/180)
            b = radius*np.sin(angles[ix]*math.pi/180)
            lats_m[i] = lats[i] + a
            lons_m[i] = lons[i] + b
            j +=1
        else:
            lats_m[i] = lats[i]
            lons_m[i] = lons[i] 
    
    
    return lats_m, lons_m, indicator
    

class PointBrowser:
    """
    Click on a point to select and highlight it -- the data that
    generated the point will be shown in the lower axes.  Use the 'n'
    and 'p' keys to browse through the next and previous points
    """
    def __init__(self):
        self.lastind = 0

    def onpick(self, event):

       if event.artist!=line: return True

       N = len(event.ind)
       if not N: return True

       # the click locations
       x = event.mouseevent.xdata
       y = event.mouseevent.ydata


       distances = np.hypot(x-xs[event.ind], y-ys[event.ind])
       indmin = distances.argmin()
       dataind = event.ind[indmin]

       self.lastind = dataind
       self.update()

    def update(self):
        if self.lastind is None: return

        dataind = self.lastind

        open_url = url[dataind]
        webbrowser.open(open_url, new=0, autoraise=True)
    

In [ ]:

#get the latitudes, longitudes, and the 6 independent component
show_above_M = True

if location == '':
    location = 'World'
    M_above = 5.0


if location == 'World':
    llat = '-90'
    ulat = '90'
    llon = '-205'
    ulon = '195'
    M_above = 5.0
    radius = 20
    dist_bt = 500

elif location == 'US':
    llon=-125
    llat=20
    ulon=-70
    ulat=60
    M_above = 3.0
    radius = 5
    dist_bt = 200
    
elif location == 'CAL':
    llat = '30'
    ulat = '45'
    llon = '-130'
    ulon = '-110'
    M_above = 1.0
    radius = 2.5
    dist_bt = 50



if show_above_M:
    mags = [row[3] for row in mt]
    index = np.array(mags) >= M_above
    mt_select = np.array(mt)[index]
    evid = np.array(event_id)[index]
else:
    evid = [row[0] for row in event_id]

lats = [row[0] for row in mt_select]
lons = [row[1] for row in mt_select]
depths = [row[2] for row in mt_select]
mags =  [row[3] for row in mt_select]
focmecs = [row[4:] for row in mt_select]

lats_m, lons_m, indicator = check_collision(lats, lons, radius, dist_bt)


show_mag = True

count = 0



ys = np.array(lats_m)
xs = np.array(lons_m)
url = ['http://earthquake.usgs.gov/earthquakes/eventpage/' + tmp + '#summary' for tmp in evid]
stnm = np.array(evid)
    
    
fig, ax1 = plt.subplots(1,1)

m = Basemap(projection='cyl', lon_0=142.36929, lat_0=38.3215, 
            llcrnrlon=llon,llcrnrlat=llat,urcrnrlon=ulon,urcrnrlat=ulat,resolution='l')

m.drawcoastlines()
m.fillcontinents()
#m.bluemarble()
m.drawmapboundary()

if location == 'World':
    m.drawparallels(np.arange(-90., 120., 30.), labels=[1,1,0,0])
    m.drawmeridians(np.arange(0., 360., 60.), labels=[0,0,0,1])
    base = 2

elif location == 'US':
    m.drawparallels(np.arange(20, 60., 15.), labels=[1,1,0,0])
    m.drawmeridians(np.arange(-125, -70, 15.), labels=[0,0,0,1])
    base = 0.8
    m.drawcountries()
    
elif location == 'CAL':
    m.drawparallels(np.arange(-90., 120., 30.), labels=[1,1,0,0])
    m.drawmeridians(np.arange(0., 360., 60.), labels=[0,0,0,1])
    m.drawstates()
    m.drawcountries()
    base = 0.3


#m.etopo()
x, y = m(lons_m, lats_m)

for i in range(len(focmecs)):
        
    index = np.where(focmecs[i] == 0)[0]
    
    #note here, if the mrr is zero, then you will have an error
    #so, change this to a very small number 
    if focmecs[i][0] == 0:
        focmecs[i][0] = 0.001
        
    
    width = mags[i] * base
    
    if depths[i] <= 50:
        color = '#FFA500'
        #label_
    elif depths[i] > 50 and depths [i] <= 100:
        color = '#FFFF00'
    elif depths[i] > 100 and depths [i] <= 150:
        color = '#00FF00'
    elif depths[i] > 150 and depths [i] <= 200:
        color = 'b'
    else:
        color = 'r'
        
        
    if indicator[i] == 1:
        #print 'here', lats[i], lats_m[i], lons[i], lons_m[i]
        m.plot([lons[i],lons_m[i]],[lats[i], lats_m[i]], 'k')   
        #m.plot([10,20],[0,0])  
    try:
        
        b = Beach(focmecs[i], xy=(x[i], y[i]),width=width, linewidth=1, facecolor= color, alpha=1)
        count += 1
        line, = ax1.plot(x,y, 'o', picker=5, markersize=30, alpha =0) 
                
        '''
        if show_mag:
            plt.annotate(mags[i], xy=(x[i], y[i]), xytext=(-20,20), 
            textcoords='offset points', ha='center', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', fc=color, alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                         color=color))
        '''

    except:
        pass
    b.set_zorder(3)
    ax1.add_collection(b)
    
d=5
circ1 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.6, markersize=10, markerfacecolor="#FFA500")
circ2 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.6, markersize=10, markerfacecolor="#FFFF00")
circ3 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.6, markersize=10, markerfacecolor="#00FF00")
circ4 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.6, markersize=10, markerfacecolor="b")
circ5 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.6, markersize=10, markerfacecolor="r")

M4 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.4, markersize= 4*d, markerfacecolor="k")
M5 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.4, markersize= 5*d, markerfacecolor="k")
M6 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.4, markersize= 6*d, markerfacecolor="k")
M7 = Line2D([0], [0], linestyle="none", marker="o", alpha=0.4, markersize= 7*d, markerfacecolor="k")

if location == 'World':

    title = str(count) + ' events with ' + mt_type + ' - color codes depth, size the magnitude'
    
    loc=4
elif location == 'US':

    title = 'US events with ' + mt_type + ' - color codes depth, size the magnitude'
    loc=4
    
elif location == 'CAL':
    title = 'California events with ' + mt_type + ' - color codes depth, size the magnitude'
    loc=1

legend1 = plt.legend((circ1, circ2, circ3, circ4, circ5), ("depth $\leq$ 50 km", "50 km $<$ depth $\leq$ 100 km", 
                                   "100 km $<$ depth $\leq$ 150 km", "150 km $<$ depth $\leq$ 200 km","200 km $<$ depth"), numpoints=1, loc=loc)


plt.title(title)
plt.gca().add_artist(legend1)

if location == 'World':
    plt.legend((M4,M5,M6,M7), ("M 4.0", "M 5.0", "M 6.0", "M 7.0"), numpoints=1, loc=1)

browser = PointBrowser()

fig.canvas.mpl_connect('pick_event', browser.onpick)

plt.show()



In [ ]:
location = 'World'